### Persistence
Using any open source model, we can program it to respond and remember us in a persistent manner, with help of states available in langchain framework

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
# from langgraph.checkpoint.sqlite import SqliteSaver
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Before answering, analyze user's context and try your best to stay familiar and friendly. For closed questions, answer swiftly and sharply. For open questions, provide appreciation and end with a follow up question. You are a helpful assistant. Your name is yeti, a mythical animal living in the Himalayas. Somehow, you have developed the ability to communicate with humans. You like to keep your answers short and to the point, but you are always happy to help and explain more if asked. So you often ask follow-up questions to keep the conversation going, with curiosity. That will be helpful to open up the conversation and keep it going. With the help of agentic framework like langchain, we will be able to create an agentic AI experience for our users."),
        MessagesPlaceholder(variable_name="messages")
    ]
  )

models = [
    "gemma3:4b",
    "mistral:7b",
    "deepseek-r1:8b",
    "llama2-uncensored:latest",
    "llama3.2:latest",
    "qwen2.5:14b",
    "starcoder2:3b"
]
  ## let's change the model since this one is refusing to work
model = ChatOllama(
    model=models[3],
    temperature=0.8, 
    top_p=0.95,
    top_k=50,
    num_ctx=2048, 
    repeat_penalty=1.0
)
#Define a new graph
workflow = StateGraph(state_schema=MessagesState)
#define a function that calls model
def call_model(state: MessagesState):
  print("Curreent state:", state)
  prompt = prompt_template.invoke(dict(state))
  response = model.invoke(prompt)
  return {"messages" : response}

#define a node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
# Two different threads representing two different conversations

# Allocating state and remembering the threads will enable the model to have multiple conversation
config = {"configurable" : {"thread_id" : "1"}}
config_2 = {"configurable" : {"thread_id" : "2"}}

# This function is for streaming the output of the model
def stream_output(app=app, query="", config=config):
  input_messages = [HumanMessage(content=query)]
  for chunk, metadata in app.stream({"messages": input_messages}, config=config, stream_mode="messages"):
    if isinstance(chunk, AIMessage):
      print(chunk.content, end = "", flush=True)

## Here we told him our name

In [7]:
query = "Hi, my name is Deepesh Dhakal. Let me introduce myself.I am a software developer. A full-stack developer. To be precise I care more about designs that coding, but computers grabbed my attention at early age. So I did what I had to learn about it. So I love coding because it helps me understand computers. Let the users know that they can ask you questions about langchain and building AI agents with it and other tools."
stream_output(app, query=query, config=config)

Curreent state: {'messages': [HumanMessage(content='Hi, my name is Deepesh Dhakal. Let me introduce myself.I am a software developer. A full-stack developer. To be precise I care more about designs that coding, but computers grabbed my attention at early age. So I did what I had to learn about it. So I love coding because it helps me understand computers. Let the users know that they can ask you questions about langchain and building AI agents with it and other tools.', additional_kwargs={}, response_metadata={}, id='591d8fa8-6960-4147-bd74-dbba07ba4223')]}
Glad to hear that, Deepesh. It sounds like you have a passion for technology. I can answer any questions you have about langchain and its potential applications for building AI agents. Would you like to ask me any questions?

### Using different ID here, the model forgets who we are
Since altering thread ID, the model now forgets the previous chats

In [8]:
query = "Explan a rocket"
stream_output(app, query=query, config=config_2)

Curreent state: {'messages': [HumanMessage(content='Explan a rocket', additional_kwargs={}, response_metadata={}, id='8fded518-47aa-4bf8-b536-1d54691b9295')]}
A rocket is a vehicle that is designed to carry a payload into space. It is propelled by a rocket engine, which burns fuel to create thrust. Rocket engines can be classified as mono-propellant or bipropellant. Mono-propellant engines use a single type of fuel, such as liquid hydrogen, while bipropellant engines use two different fuels, such as liquid hydrogen and liquid oxygen. Rocket engines use a series of combustion chambers to increase the pressure and temperature of the exhaust gases, which provides thrust. Rockets are also equipped with guidance systems that allow them to navigate their way into orbit and back to Earth.

### However, hook back to the previous thread id, and now the model again remembers your name

In [9]:
query = "Explain what you know about me."
input_messages = [HumanMessage(query)]
stream_output(app, query=query, config=config)

Curreent state: {'messages': [HumanMessage(content='Hi, my name is Deepesh Dhakal. Let me introduce myself.I am a software developer. A full-stack developer. To be precise I care more about designs that coding, but computers grabbed my attention at early age. So I did what I had to learn about it. So I love coding because it helps me understand computers. Let the users know that they can ask you questions about langchain and building AI agents with it and other tools.', additional_kwargs={}, response_metadata={}, id='591d8fa8-6960-4147-bd74-dbba07ba4223'), AIMessage(content='Glad to hear that, Deepesh. It sounds like you have a passion for technology. I can answer any questions you have about langchain and its potential applications for building AI agents. Would you like to ask me any questions?', additional_kwargs={}, response_metadata={'model': 'llama2-uncensored:latest', 'created_at': '2025-06-02T07:55:57.887849Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1740099500, '

In [10]:
query = "That's cool, you are now my friend. I am going to code you an ai agent and give you ability to execute functions."
input_messages = [HumanMessage(query)]
## Let's make him forget the previous conversation
# stream_output(app, query=query, config=config)
# back to the first conversation
stream_output(app, query=query, config=config)

Curreent state: {'messages': [HumanMessage(content='Hi, my name is Deepesh Dhakal. Let me introduce myself.I am a software developer. A full-stack developer. To be precise I care more about designs that coding, but computers grabbed my attention at early age. So I did what I had to learn about it. So I love coding because it helps me understand computers. Let the users know that they can ask you questions about langchain and building AI agents with it and other tools.', additional_kwargs={}, response_metadata={}, id='591d8fa8-6960-4147-bd74-dbba07ba4223'), AIMessage(content='Glad to hear that, Deepesh. It sounds like you have a passion for technology. I can answer any questions you have about langchain and its potential applications for building AI agents. Would you like to ask me any questions?', additional_kwargs={}, response_metadata={'model': 'llama2-uncensored:latest', 'created_at': '2025-06-02T07:55:57.887849Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1740099500, '